# Genetic Algorithm

In [1]:
import numpy as np

def fitness_function(individual, values, weights, max_weight=25):
    total_value = np.sum(values[individual == 1])
    total_weight = np.sum(weights[individual == 1])
    
    if total_weight > max_weight:
        return 0
    return total_value

def initialize_population(pop_size, n_items):
    return np.random.randint(2, size=(pop_size, n_items))

def tournament_selection(population, fitness_scores, tournament_size=3):
    tournament_indices = np.random.choice(len(population), tournament_size)
    tournament_fitness = fitness_scores[tournament_indices]
    winner_index = tournament_indices[np.argmax(tournament_fitness)]
    return population[winner_index]

def crossover(parent1, parent2, crossover_rate=0.25):
    if np.random.random() > crossover_rate:
        return parent1, parent2
    
    cross_point = np.random.randint(1, len(parent1)-1)
    child1 = np.concatenate([parent1[:cross_point], parent2[cross_point:]])
    child2 = np.concatenate([parent2[:cross_point], parent1[cross_point:]])
    return child1, child2

def mutation(individual, mutation_rate=0.05):
    for i in range(len(individual)):
        if np.random.random() < mutation_rate:
            individual[i] = 1 - individual[i]
    return individual

def genetic_algorithm(values, weights, pop_size, n_generations=100, 
                     crossover_rate=0.99, mutation_rate=0.05):
    n_items = len(values)
    population = initialize_population(pop_size, n_items)
    best_solution = None
    best_fitness = 0
    
    for generation in range(n_generations):
        # Calculate fitness for all individuals
        fitness_scores = np.array([fitness_function(ind, values, weights) 
                                 for ind in population])
        
        # Keep track of best solution
        generation_best_idx = np.argmax(fitness_scores)
        if fitness_scores[generation_best_idx] > best_fitness:
            best_fitness = fitness_scores[generation_best_idx]
            best_solution = population[generation_best_idx].copy()
        
        # Create new population
        new_population = []
        
        while len(new_population) < pop_size:
            # Selection
            parent1 = tournament_selection(population, fitness_scores)
            parent2 = tournament_selection(population, fitness_scores)
            
            # Crossover
            child1, child2 = crossover(parent1, parent2, crossover_rate)
            
            # Mutation
            child1 = mutation(child1, mutation_rate)
            child2 = mutation(child2, mutation_rate)
            
            new_population.extend([child1, child2])
        
        # Ensure population size stays constant
        population = np.array(new_population[:pop_size])
        
        if (generation + 1) % 10 == 0:
            print(f"Generation {generation + 1}: Best Fitness = {best_fitness}")
    
    return best_solution, best_fitness

# Run the algorithm with the given values
V = np.array([6, 9, 5, 3, 7, 6, 8, 4, 7, 4])
W = np.array([5, 8, 6, 4, 6, 5, 9, 5, 6, 5])
pop_size = 10

best_solution, best_fitness = genetic_algorithm(V, W, pop_size)

print("\nFinal Results:")
print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)
print("Total Weight:", np.sum(W[best_solution == 1]))
print("Selected Items:", np.where(best_solution == 1)[0])

Generation 10: Best Fitness = 25
Generation 20: Best Fitness = 25
Generation 30: Best Fitness = 25
Generation 40: Best Fitness = 25
Generation 50: Best Fitness = 25
Generation 60: Best Fitness = 27
Generation 70: Best Fitness = 27
Generation 80: Best Fitness = 27
Generation 90: Best Fitness = 27
Generation 100: Best Fitness = 27

Final Results:
Best Solution: [0 1 1 0 1 1 0 0 0 0]
Best Fitness: 27
Total Weight: 25
Selected Items: [1 2 4 5]


# Quantum Genetic Algorithm (With qiskit)

In [13]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.primitives import Sampler
from qiskit.visualization import circuit_drawer

class QuantumGeneticKnapsack:
    def __init__(self, values, weights, pop_size, max_weight=25,
                 crossover_rate=0.25, mutation_rate=0.05):
        self.values = values
        self.weights = weights
        self.pop_size = pop_size
        self.n_items = len(values)
        self.max_weight = max_weight
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.sampler = Sampler()
    
    def fitness_function(self, individual):
        total_value = np.sum(self.values[individual == 1])
        total_weight = np.sum(self.weights[individual == 1])
        
        if total_weight > self.max_weight:
            return 0
        return total_value

    def quantum_initialize_population(self):
        population = []
        
        for _ in range(self.pop_size):
            qc = QuantumCircuit(self.n_items, self.n_items)
            # Apply Hadamard gates to create superposition
            qc.h(range(self.n_items))
            qc.measure(range(self.n_items), range(self.n_items))
            
            # Use Sampler primitive
            job = self.sampler.run([qc], shots=1)
            result = job.result()
            counts = result.quasi_dists[0]
            # Get the measured bitstring
            binary = format(list(counts.keys())[0], f'0{self.n_items}b')
            individual = np.array([int(bit) for bit in binary[::-1]])
            population.append(individual)
            
        return np.array(population)

    def quantum_crossover(self, parent1, parent2):
        if np.random.random() > self.crossover_rate:
            return parent1.copy(), parent2.copy()
        
        n_qubits = len(parent1)
        # Create separate quantum and classical registers
        qr = QuantumRegister(n_qubits * 2, 'q')  # Double qubits for both parents
        cr = ClassicalRegister(n_qubits * 2, 'c')
        qc = QuantumCircuit(qr, cr)
        
        # Encode parents into quantum states
        for i, (p1, p2) in enumerate(zip(parent1, parent2)):
            if p1:
                qc.x(qr[i])
            if p2:
                qc.x(qr[i + n_qubits])
        
        # Apply quantum crossover operation (controlled-SWAP)
        cross_point = np.random.randint(1, n_qubits-1)
        control_qubit = qr[0]  # Use first qubit as control
        for i in range(cross_point, n_qubits):
            target1 = qr[i]
            target2 = qr[i + n_qubits]
            # Implement CSWAP (Fredkin gate) using the specified qubits
            qc.cswap(control_qubit, target1, target2)
        
        qc.measure(qr, cr)
        
        # Use Sampler primitive
        job = self.sampler.run([qc], shots=1)
        result = job.result()
        counts = result.quasi_dists[0]
        binary = format(list(counts.keys())[0], f'0{2*n_qubits}b')
        
        # Convert result to two children
        child1 = np.array([int(bit) for bit in binary[:n_qubits][::-1]])
        child2 = np.array([int(bit) for bit in binary[n_qubits:][::-1]])
        
        return child1, child2

    def quantum_mutation(self, individual):
        n_qubits = len(individual)
        qc = QuantumCircuit(n_qubits, n_qubits)
        
        # Encode individual
        for i, bit in enumerate(individual):
            if bit:
                qc.x(i)
            
            # Apply mutation with probability
            if np.random.random() < self.mutation_rate:
                qc.h(i)  # Hadamard gate for quantum mutation
                qc.t(i)  # T gate to introduce phase
                qc.h(i)
        
        qc.measure(range(n_qubits), range(n_qubits))
        
        # Use Sampler primitive
        job = self.sampler.run([qc], shots=1)
        result = job.result()
        counts = result.quasi_dists[0]
        binary = format(list(counts.keys())[0], f'0{n_qubits}b')
        
        return np.array([int(bit) for bit in binary[::-1]])

    def run(self, n_generations=10):
        # Initialize population using quantum superposition
        population = self.quantum_initialize_population()
        best_solution = None
        best_fitness = 0
        
        for generation in range(n_generations):
            # Calculate fitness for all individuals
            fitness_scores = np.array([self.fitness_function(ind) for ind in population])
            
            # Update best solution
            generation_best_idx = np.argmax(fitness_scores)
            if fitness_scores[generation_best_idx] > best_fitness:
                best_fitness = fitness_scores[generation_best_idx]
                best_solution = population[generation_best_idx].copy()
            
            # Create new population
            new_population = []
            
            # Tournament selection and quantum operations
            while len(new_population) < self.pop_size:
                # Select parents using tournament selection
                tournament_indices = np.random.choice(len(population), 3)
                tournament_fitness = fitness_scores[tournament_indices]
                parent1 = population[tournament_indices[np.argmax(tournament_fitness)]]
                
                tournament_indices = np.random.choice(len(population), 3)
                tournament_fitness = fitness_scores[tournament_indices]
                parent2 = population[tournament_indices[np.argmax(tournament_fitness)]]
                
                # Quantum crossover
                child1, child2 = self.quantum_crossover(parent1, parent2)
                
                # Quantum mutation
                child1 = self.quantum_mutation(child1)
                child2 = self.quantum_mutation(child2)
                
                new_population.extend([child1, child2])
            
            # Update population
            population = np.array(new_population[:self.pop_size])
            
            if (generation + 1) % 1 == 0:
                print(f"Generation {generation + 1}: Best Fitness = {best_fitness}")
        
        return best_solution, best_fitness

# Example usage
if __name__ == "__main__":
    V = np.array([6, 9, 5, 3, 7, 6, 8, 4, 7, 4])
    W = np.array([5, 8, 6, 4, 6, 5, 9, 5, 6, 5])
    pop_size = 100
    
    qga = QuantumGeneticKnapsack(V, W, pop_size)
    best_solution, best_fitness = qga.run()
    
    print("\nFinal Results:")
    print("Best Solution:", best_solution)
    print("Best Fitness:", best_fitness)
    print("Total Weight:", np.sum(W[best_solution == 1]))
    print("Selected Items:", np.where(best_solution == 1)[0])

/tmp/ipykernel_2867/816258042.py:16: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  self.sampler = Sampler()


Generation 1: Best Fitness = 29
Generation 2: Best Fitness = 29
Generation 3: Best Fitness = 29
Generation 4: Best Fitness = 29
Generation 5: Best Fitness = 29
Generation 6: Best Fitness = 29
Generation 7: Best Fitness = 29
Generation 8: Best Fitness = 29
Generation 9: Best Fitness = 29
Generation 10: Best Fitness = 29

Final Results:
Best Solution: [0 1 0 0 1 1 0 0 1 0]
Best Fitness: 29
Total Weight: 25
Selected Items: [1 4 5 8]


# Quantum Genetic Algorithm (Simulation)

In [2]:
import numpy as np

def quantum_initialize_population(pop_size, n_items):
    # Initialize quantum angles (theta) between 0 and π/2
    theta = np.random.uniform(0, np.pi/2, (pop_size, n_items))
    
    # Quantum state amplitudes
    alpha = np.cos(theta)  # Probability amplitude for |0⟩
    beta = np.sin(theta)   # Probability amplitude for |1⟩
    
    return alpha, beta

def quantum_observe(alpha, beta):
    # Collapse quantum states to classical bits based on probability
    probabilities = beta ** 2  # Probability of observing |1⟩
    random_numbers = np.random.random(alpha.shape)
    return (random_numbers < probabilities).astype(int)

def quantum_rotation(alpha, beta, best_solution, delta_theta=0.01 * np.pi):
    # Quantum rotation gate to move quantum states toward better solutions
    theta = np.arctan2(beta, alpha)
    
    # Determine rotation direction based on best solution
    direction = np.where(best_solution == 1, 1, -1)
    
    # Apply rotation
    new_theta = theta + direction * delta_theta
    
    # Update quantum amplitudes
    new_alpha = np.cos(new_theta)
    new_beta = np.sin(new_theta)
    
    return new_alpha, new_beta

def quantum_interference(alpha, beta):
    # Simulate quantum interference between states
    # Using a simplified interference model based on phase relationships
    phase = np.random.uniform(0, 2*np.pi, alpha.shape)
    alpha_interfered = alpha * np.cos(phase)
    beta_interfered = beta * np.sin(phase)
    
    # Normalize to maintain probability sum = 1
    norm = np.sqrt(alpha_interfered**2 + beta_interfered**2)
    return alpha_interfered/norm, beta_interfered/norm

def fitness_function(individual, values, weights, max_weight=25):
    total_value = np.sum(values[individual == 1])
    total_weight = np.sum(weights[individual == 1])
    
    if total_weight > max_weight:
        return 0
    return total_value

def quantum_genetic_algorithm(values, weights, pop_size, n_generations=20):
    n_items = len(values)
    
    # Initialize quantum population
    alpha, beta = quantum_initialize_population(pop_size, n_items)
    
    best_solution = None
    best_fitness = 0
    
    for generation in range(n_generations):
        # Quantum interference to create superposition
        alpha, beta = quantum_interference(alpha, beta)
        
        # Observe quantum states to get classical population
        classical_population = quantum_observe(alpha, beta)
        
        # Evaluate fitness in parallel using matrix operations
        fitness_scores = np.zeros(pop_size)
        for i in range(pop_size):
            fitness_scores[i] = fitness_function(classical_population[i], values, weights)
        
        # Update best solution
        generation_best_idx = np.argmax(fitness_scores)
        if fitness_scores[generation_best_idx] > best_fitness:
            best_fitness = fitness_scores[generation_best_idx]
            best_solution = classical_population[generation_best_idx].copy()
        
        # Quantum rotation toward best solution
        alpha, beta = quantum_rotation(alpha, beta, best_solution)
        
        # Apply quantum mutation through phase perturbation
        if np.random.random() < 0.05:  # mutation rate
            phase_noise = np.random.normal(0, 0.1, alpha.shape)
            alpha = alpha * np.cos(phase_noise)
            beta = beta * np.sin(phase_noise)
            # Renormalize
            norm = np.sqrt(alpha**2 + beta**2)
            alpha = alpha/norm
            beta = beta/norm
        
        if (generation + 1) % 1 == 0:
            print(f"Generation {generation + 1}: Best Fitness = {best_fitness}")
    
    return best_solution, best_fitness

# Run the algorithm with the given values
V = np.array([6, 9, 5, 3, 7, 6, 8, 4, 7, 4])
W = np.array([5, 8, 6, 4, 6, 5, 9, 5, 6, 5])
pop_size = 100

best_solution, best_fitness = quantum_genetic_algorithm(V, W, pop_size)

print("\nFinal Results:")
print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)
print("Total Weight:", np.sum(W[best_solution == 1]))
print("Selected Items:", np.where(best_solution == 1)[0])

Generation 1: Best Fitness = 27.0
Generation 2: Best Fitness = 28.0
Generation 3: Best Fitness = 28.0
Generation 4: Best Fitness = 28.0
Generation 5: Best Fitness = 28.0
Generation 6: Best Fitness = 29.0
Generation 7: Best Fitness = 29.0
Generation 8: Best Fitness = 29.0
Generation 9: Best Fitness = 29.0
Generation 10: Best Fitness = 29.0
Generation 11: Best Fitness = 29.0
Generation 12: Best Fitness = 29.0
Generation 13: Best Fitness = 29.0
Generation 14: Best Fitness = 29.0
Generation 15: Best Fitness = 29.0
Generation 16: Best Fitness = 29.0
Generation 17: Best Fitness = 29.0
Generation 18: Best Fitness = 29.0
Generation 19: Best Fitness = 29.0
Generation 20: Best Fitness = 29.0
Generation 21: Best Fitness = 29.0
Generation 22: Best Fitness = 29.0
Generation 23: Best Fitness = 29.0
Generation 24: Best Fitness = 29.0
Generation 25: Best Fitness = 29.0
Generation 26: Best Fitness = 29.0
Generation 27: Best Fitness = 29.0
Generation 28: Best Fitness = 29.0
Generation 29: Best Fitness =

# Simplified Quantum Genetic Algorithm (Simulation / No crossover)

In [3]:
import numpy as np

def initialize_quantum_population(pop_size, n_items):
    # Initialize with equal superposition (π/4 gives 50-50 chance)
    return np.ones((pop_size, n_items)) * np.pi/4

def measure_population(quantum_angles):
    # Convert quantum angles to classical bits based on probability
    probabilities = np.sin(quantum_angles) ** 2
    return (np.random.random(quantum_angles.shape) < probabilities).astype(int)

def evaluate_fitness(solution, values, weights, max_weight=25):
    total_value = np.sum(values[solution == 1])
    total_weight = np.sum(weights[solution == 1])
    return 0 if total_weight > max_weight else total_value

def update_quantum_angles(quantum_angles, best_solution, delta=0.01 * np.pi):
    # Rotate quantum angles towards best solution
    direction = np.where(best_solution == 1, 1, -1)
    return quantum_angles + direction * delta

def quantum_genetic_algorithm(values, weights, pop_size, generations=100):
    n_items = len(values)
    
    # Initialize quantum population
    quantum_angles = initialize_quantum_population(pop_size, n_items)
    
    best_solution = None
    best_fitness = 0
    
    for gen in range(generations):
        # Measure quantum states to get classical solutions
        classical_solutions = measure_population(quantum_angles)
        
        # Evaluate all solutions
        fitness_scores = np.array([evaluate_fitness(sol, values, weights) 
                                 for sol in classical_solutions])
        
        # Update best solution
        current_best_idx = np.argmax(fitness_scores)
        if fitness_scores[current_best_idx] > best_fitness:
            best_fitness = fitness_scores[current_best_idx]
            best_solution = classical_solutions[current_best_idx].copy()
        
        # Update quantum angles towards best solution
        quantum_angles = update_quantum_angles(quantum_angles, best_solution)
        
        # Simple quantum mutation
        if np.random.random() < 0.05:  # mutation rate
            mutation_mask = np.random.random(quantum_angles.shape) < 0.1
            quantum_angles[mutation_mask] += np.random.normal(0, 0.1, np.sum(mutation_mask))
        
        if (gen + 1) % 10 == 0:
            print(f"Generation {gen + 1}: Best Fitness = {best_fitness}")
    
    return best_solution, best_fitness

# Test the algorithm
V = np.array([6, 9, 5, 3, 7, 6, 8, 4, 7, 4])
W = np.array([5, 8, 6, 4, 6, 5, 9, 5, 6, 5])
pop_size = 5

best_solution, best_fitness = quantum_genetic_algorithm(V, W, pop_size)

print("\nResults:")
print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)
print("Total Weight:", np.sum(W[best_solution == 1]))
print("Selected Items:", np.where(best_solution == 1)[0])

Generation 10: Best Fitness = 26
Generation 20: Best Fitness = 28
Generation 30: Best Fitness = 28
Generation 40: Best Fitness = 28
Generation 50: Best Fitness = 29
Generation 60: Best Fitness = 29
Generation 70: Best Fitness = 29
Generation 80: Best Fitness = 29
Generation 90: Best Fitness = 29
Generation 100: Best Fitness = 29

Results:
Best Solution: [1 1 0 0 1 0 0 0 1 0]
Best Fitness: 29
Total Weight: 25
Selected Items: [0 1 4 8]


# Simplified Quantum Genetic Algorithm (Simulation / with crossover)

In [4]:
import numpy as np

def initialize_quantum_population(pop_size, n_items):
    # Initialize with equal superposition (π/4 gives 50-50 chance)
    return np.ones((pop_size, n_items)) * np.pi/4

def measure_population(quantum_angles):
    # Convert quantum angles to classical bits based on probability
    probabilities = np.sin(quantum_angles) ** 2
    return (np.random.random(quantum_angles.shape) < probabilities).astype(int)

def evaluate_fitness(solution, values, weights, max_weight=25):
    total_value = np.sum(values[solution == 1])
    total_weight = np.sum(weights[solution == 1])
    return 0 if total_weight > max_weight else total_value

def quantum_crossover(parent1_angles, parent2_angles, crossover_rate=0.25):
    if np.random.random() > crossover_rate:
        return parent1_angles.copy(), parent2_angles.copy()
    
    # Quantum superposition crossover
    crossover_point = np.random.randint(1, len(parent1_angles)-1)
    
    # Create superposition of parent states
    child1_angles = np.zeros_like(parent1_angles)
    child2_angles = np.zeros_like(parent2_angles)
    
    # Mix quantum angles
    mixing_ratio = np.random.random()  # Random superposition weight
    child1_angles[:crossover_point] = parent1_angles[:crossover_point]
    child1_angles[crossover_point:] = mixing_ratio * parent1_angles[crossover_point:] + \
                                    (1 - mixing_ratio) * parent2_angles[crossover_point:]
    
    child2_angles[:crossover_point] = parent2_angles[:crossover_point]
    child2_angles[crossover_point:] = mixing_ratio * parent2_angles[crossover_point:] + \
                                    (1 - mixing_ratio) * parent1_angles[crossover_point:]
    
    return child1_angles, child2_angles

def update_quantum_angles(quantum_angles, best_solution, delta=0.01 * np.pi):
    # Rotate quantum angles towards best solution
    direction = np.where(best_solution == 1, 1, -1)
    return quantum_angles + direction * delta

def tournament_selection(quantum_angles, fitness_scores, tournament_size=3):
    tournament_idx = np.random.choice(len(quantum_angles), tournament_size)
    winner_idx = tournament_idx[np.argmax(fitness_scores[tournament_idx])]
    return quantum_angles[winner_idx]

def quantum_genetic_algorithm(values, weights, pop_size, generations=100):
    n_items = len(values)
    
    # Initialize quantum population
    quantum_angles = initialize_quantum_population(pop_size, n_items)
    
    best_solution = None
    best_fitness = 0
    
    for gen in range(generations):
        # Measure quantum states to get classical solutions
        classical_solutions = measure_population(quantum_angles)
        
        # Evaluate all solutions
        fitness_scores = np.array([evaluate_fitness(sol, values, weights) 
                                 for sol in classical_solutions])
        
        # Update best solution
        current_best_idx = np.argmax(fitness_scores)
        if fitness_scores[current_best_idx] > best_fitness:
            best_fitness = fitness_scores[current_best_idx]
            best_solution = classical_solutions[current_best_idx].copy()
        
        # Create new population through quantum operators
        new_quantum_angles = []
        
        while len(new_quantum_angles) < pop_size:
            # Tournament selection
            parent1 = tournament_selection(quantum_angles, fitness_scores)
            parent2 = tournament_selection(quantum_angles, fitness_scores)
            
            # Quantum crossover
            child1_angles, child2_angles = quantum_crossover(parent1, parent2)
            
            # Quantum mutation
            if np.random.random() < 0.05:  # mutation rate
                mutation_mask = np.random.random(child1_angles.shape) < 0.1
                child1_angles[mutation_mask] += np.random.normal(0, 0.1, np.sum(mutation_mask))
                child2_angles[mutation_mask] += np.random.normal(0, 0.1, np.sum(mutation_mask))
            
            new_quantum_angles.extend([child1_angles, child2_angles])
        
        # Update population
        quantum_angles = np.array(new_quantum_angles[:pop_size])
        
        # Rotate towards best solution
        quantum_angles = update_quantum_angles(quantum_angles, best_solution)
        
        if (gen + 1) % 10 == 0:
            print(f"Generation {gen + 1}: Best Fitness = {best_fitness}")
    
    return best_solution, best_fitness

# Test the algorithm
V = np.array([6, 9, 5, 3, 7, 6, 8, 4, 7, 4])
W = np.array([5, 8, 6, 4, 6, 5, 9, 5, 6, 5])
pop_size = 100

best_solution, best_fitness = quantum_genetic_algorithm(V, W, pop_size)

print("\nResults:")
print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)
print("Total Weight:", np.sum(W[best_solution == 1]))
print("Selected Items:", np.where(best_solution == 1)[0])

Generation 10: Best Fitness = 29
Generation 20: Best Fitness = 29
Generation 30: Best Fitness = 29
Generation 40: Best Fitness = 29
Generation 50: Best Fitness = 29
Generation 60: Best Fitness = 29
Generation 70: Best Fitness = 29
Generation 80: Best Fitness = 29
Generation 90: Best Fitness = 29
Generation 100: Best Fitness = 29

Results:
Best Solution: [1 1 0 0 1 0 0 0 1 0]
Best Fitness: 29
Total Weight: 25
Selected Items: [0 1 4 8]
